In [2]:
import requests
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
#import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
#from folium import Choropleth
from folium.plugins import HeatMap
import datetime
from plotly.offline import init_notebook_mode, plot, iplot
import plotly as py
init_notebook_mode(connected=True) 
import plotly.graph_objs as go
import pickle

In [ ]:
def run_query(year_start, year_end):
    months = ["01","02","03","04","05","06","07","08","09","10","11","12"]
    df = []
    for y in range(year_start, year_end):
        for m in months:
            x =requests.get(f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={str(y)}-{m}-01&endtime={str(y)}-{m}-31')
            df.append(x)
    return df

df_1920_1960 = run_query(1920, 1960)
df_1960_1980 = run_query(1960, 1980)
df_1980_2000 = run_query(1980, 2000)
df_2000_2020 = run_query(2000, 2023)

In [5]:
pickle_in = open("1920-1960.pickle", "rb")
df_1920_to_1960 = pickle.load(pickle_in)

pickle_in = open("1960-1980.pickle", "rb")
df_1960_to_1980 = pickle.load(pickle_in)

pickle_in = open("1980-2000.pickle", "rb")
df_1980_to_2000 = pickle.load(pickle_in)

pickle_in = open("2000-2023.pickle", "rb")
df_2000_to_2023 = pickle.load(pickle_in)

In [6]:
df_list = []

#At the time of this code, predata was 1920-1960. Below I use postdata which was 1960-2022

list_of_year_ranges = [df_1920_to_1960,df_1960_to_1980,df_1980_to_2000,df_2000_to_2023]

for i in list_of_year_ranges:
    for j in i:   
        try:
            x = j.json()
            y = json_normalize(x,'features')
            z = pd.DataFrame(y)
            df_list.append(z)
        except:
            continue
final_df = pd.concat(df_list)

In [7]:
final_df.shape

(4077518, 30)

In [ ]:
final_df.dtypes

In [8]:
final_df['Time'] = pd.to_datetime((final_df['properties.time']), unit = 'ms')

In [ ]:
#final_df['Time'].isnull().sum()

In [9]:
final_df[['Longitude','Latitude','Depth']] = pd.DataFrame(final_df['geometry.coordinates'].tolist(),index= final_df.index)

In [10]:
column_names = ['Type','ID','mag','place','time(ms)','updated','tz','url','detail','felt','cdi','mmi','alert','status','tsunami','sig','net','code','ids','sources','types','nst','dmin','rms','gap','magType','type','title','geometry.type','geometry.coordinates', 'Time','Longitude','Latitude','Depth'
]
final_df.set_axis(column_names, axis=1,inplace=True)

In [ ]:
final_df.Time.groupby(final_df["Time"].dt.year).count().plot(kind="bar")
# We need to normalize? this data set and also adjust the scale and x labels.

In [ ]:
# Let's look at magnitudes of earthquakes over time.
# plotting a lineplot with magnitudes with respect to dates
plt.figure(figsize=(12,8))
# We should change this to .year instead when we have the full data
Time_series = sns.lineplot(x=final_df["Time"].dt.year,y="mag",data=final_df, color="#997e48")
Time_series.set_title("Average Magnitude of Earthquakes each Year ", color="#58508d")
Time_series.set_ylabel("Magnitude", color="#58508d")
Time_series.set_xlabel("Date", color="#58508d")

In [ ]:
tectonic_plates = pd.read_csv("tectonic_plate_boundaries.csv" )

In [ ]:
#Ploting the tectonic plate's boundaries 
tectonic = folium.Map(tiles="cartodbpositron", zoom_start=5)

plates = list(tectonic_plates["plate"].unique())
for plate in plates:
    plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
    lats = plate_vals["lat"].values
    lons = plate_vals["lon"].values
    points = list(zip(lats, lons))
    indexes = [None] + [i + 1 for i, x in enumerate(points) if i < len(points) - 1 and abs(x[1] - points[i + 1][1]) > 300] + [None]
    for i in range(len(indexes) - 1):
        folium.vector_layers.PolyLine(points[indexes[i]:indexes[i+1]], popup=plate, color="#58508d", fill=False, ).add_to(tectonic)

tectonic

In [ ]:
#Plottong the plates along with the earthquake
tectonic_quake = folium.Map(tiles="cartodbpositron", zoom_start=5)
gradient = {.33: "#7a5195", .66: "#ef5675", 1: "#ffa600"} 
plates = list(tectonic_plates["plate"].unique())
for plate in plates:
    plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
    lats = plate_vals["lat"].values
    lons = plate_vals["lon"].values
    points = list(zip(lats, lons))
    indexes = [None] + [i + 1 for i, x in enumerate(points) if i < len(points) - 1 and abs(x[1] - points[i + 1][1]) > 300] + [None]
    for i in range(len(indexes) - 1):
        folium.vector_layers.PolyLine(points[indexes[i]:indexes[i+1]], popup=plate, fill=False,  color="#58508d").add_to(tectonic_quake)
        HeatMap(data=final_df[["Latitude", "Longitude"]],hue="properties.mag",min_opacity=0.5,radius=1,gradient=gradient).add_to(tectonic_quake)
tectonic_quake

In [ ]:
# Create a base map with plate boundaries and deapth 
Mag_tectonics  = folium.Map(tiles="cartodbpositron", zoom_start=5)
gradient = {.33: "#628d82", .66: "#a3c5bf", 1: "#eafffd"} 
plates = list(tectonic_plates["plate"].unique())
for plate in plates:
    plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
    lats = plate_vals["lat"].values
    lons = plate_vals["lon"].values
    points = list(zip(lats, lons))
    indexes = [None] + [i + 1 for i, x in enumerate(points) if i < len(points) - 1 and abs(x[1] - points[i + 1][1]) > 300] + [None]
    for i in range(len(indexes) - 1):
        folium.vector_layers.PolyLine(points[indexes[i]:indexes[i+1]], fill=False,  color="#58508d").add_to(Mag_tectonics)
        
def colormag(val):
            if val < 5.9:
                return "#ffcf6a"
            elif val < 6.5:
                return "#fb8270"
            else:
                return "#bc5090"
            
for i in range(0,len(df)):
    folium.Circle(location=[df.iloc[i]["Latitude"], df.iloc[i]["Longitude"]],radius=2000, color=colormag(df.iloc[i]["mag"])).add_to(Mag_tectonics)
    
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Magnitude Scale </div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#ffcf6a;opacity:1;'></span>values less than 5.9</li>
    <li><span style='background:#fb8270;opacity:1;'></span>values less than 6.5</li>
    <li><span style='background:#bc5090;opacity:1;'></span>values less than 6.5</li>

  </ul>
</div>
</div>

</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
        }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

Mag_tectonics.get_root().add_child(macro)

Mag_tectonics

In [ ]:
# Create a base map with plate boundaries and deapth 
Depth_tectonics  = folium.Map(tiles="cartodbpositron", zoom_start=5)
gradient = {.33: "#628d82", .66: "#a3c5bf", 1: "#eafffd"} 
plates = list(tectonic_plates["plate"].unique())
for plate in plates:
    plate_vals = tectonic_plates[tectonic_plates["plate"] == plate]
    lats = plate_vals["lat"].values
    lons = plate_vals["lon"].values
    points = list(zip(lats, lons))
    indexes = [None] + [i + 1 for i, x in enumerate(points) if i < len(points) - 1 and abs(x[1] - points[i + 1][1]) > 300] + [None]
    for i in range(len(indexes) - 1):
        folium.vector_layers.PolyLine(points[indexes[i]:indexes[i+1]],fill=False,  color="#58508d").add_to(Depth_tectonics)
        
def colordepth(val):
            if val < 50:
                return "#ffcf6a"
            elif val < 100:
                return "#fb8270"
            else:
                return "#bc5090"
            
for i in range(0,len(df)):
    folium.Circle(location=[df.iloc[i]["Latitude"], df.iloc[i]["Longitude"]],radius=2000, color=colordepth(df.iloc[i]["Depth"])).add_to(Depth_tectonics)

#A bit of extra code to get legent 
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Depth Scale </div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#ffcf6a;opacity:1;'></span>Depth less than 50</li>
    <li><span style='background:#fb8270;opacity:1;'></span>Depth less than 100</li>
    <li><span style='background:#bc5090;opacity:1;'></span>Depth more than 100</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

Depth_tectonics.get_root().add_child(macro)

Depth_tectonics
